## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import G_key

# Configure gmaps API key
gmaps.configure(api_key=G_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Katsuura,JP,2021-11-27 16:54:15,35.1333,140.3000,46.51,55,27,9.60,scattered clouds
1,1,Priiskovyy,RU,2021-11-27 16:54:16,51.9333,116.6333,9.91,96,100,0.78,overcast clouds
2,2,Filingue,NE,2021-11-27 16:54:16,14.3521,3.3168,90.95,10,82,7.85,broken clouds
3,3,Busselton,AU,2021-11-27 16:54:16,-33.6500,115.3333,64.87,67,0,6.38,clear sky
4,4,Bambous Virieux,MU,2021-11-27 16:54:17,-20.3428,57.7575,79.05,73,20,3.44,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Bambous Virieux,MU,2021-11-27 16:54:17,-20.3428,57.7575,79.05,73,20,3.44,few clouds
6,6,Suez,EG,2021-11-27 16:54:18,29.9737,32.5263,71.94,33,7,7.96,clear sky
9,9,Hermanus,ZA,2021-11-27 16:54:19,-34.4187,19.2345,67.32,65,98,18.77,overcast clouds
13,13,Castro,BR,2021-11-27 16:49:32,-24.7911,-50.0119,79.38,47,4,7.05,clear sky
20,20,Carnarvon,AU,2021-11-27 16:49:32,-24.8667,113.6333,78.87,47,0,17.27,clear sky
21,21,Rafai,CF,2021-11-27 16:54:23,4.9500,23.9167,78.33,42,38,2.98,scattered clouds
22,22,Nabire,ID,2021-11-27 16:54:23,-3.3667,135.4833,74.98,96,100,3.98,overcast clouds
23,23,Merritt Island,US,2021-11-27 16:54:24,28.5392,-80.6720,66.22,61,1,11.50,clear sky
24,24,Chuy,UY,2021-11-27 16:51:09,-33.6971,-53.4616,72.48,71,99,11.16,overcast clouds
28,28,Caravelas,BR,2021-11-27 16:54:25,-17.7125,-39.2481,75.88,78,92,6.64,moderate rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()
#preferred_cities_df.isnull().sum()

City_ID                196
City                   196
Country                195
Date                   196
Lat                    196
Lng                    196
Max Temp               196
Humidity               196
Cloudiness             196
Wind Speed             196
Current Description    196
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df
len(clean_df)

195

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Bambous Virieux,MU,79.05,few clouds,-20.3428,57.7575,
6,Suez,EG,71.94,clear sky,29.9737,32.5263,
9,Hermanus,ZA,67.32,overcast clouds,-34.4187,19.2345,
13,Castro,BR,79.38,clear sky,-24.7911,-50.0119,
20,Carnarvon,AU,78.87,clear sky,-24.8667,113.6333,
21,Rafai,CF,78.33,scattered clouds,4.9500,23.9167,
22,Nabire,ID,74.98,overcast clouds,-3.3667,135.4833,
23,Merritt Island,US,66.22,clear sky,28.5392,-80.6720,
24,Chuy,UY,72.48,overcast clouds,-33.6971,-53.4616,
28,Caravelas,BR,75.88,moderate rain,-17.7125,-39.2481,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": G_key
}
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame    
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
#     hotel_df.loc[index, "Hotel Name"] = len(hotels["results"][0]["name"])
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
# clean_hotel_df = hotel_df.dropna()
# clean_hotel_df.head(15)
# clean_hotel_df.count()
# hotel_df.at[6,'Hotel Name']=''
# hotel_df
# #clean_hotel_df = hotel_df.dropna()


# clean_hotel_df = hotel_df.dropna(axis='index', how="any")
# clean_hotel_df
hotel_df.rename(columns={"Hotel Name": "HotelName"}, inplace=True)
hotel_df
hotel_df.query("HotelName==''")
hotel_df = hotel_df.query("HotelName!=''")
hotel_df
clean_hotel_df = hotel_df.dropna(axis='index', how="any")
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,HotelName
4,Bambous Virieux,MU,79.05,few clouds,-20.3428,57.7575,Casa Tia Villa
6,Suez,EG,71.94,clear sky,29.9737,32.5263,Green House
9,Hermanus,ZA,67.32,overcast clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
13,Castro,BR,79.38,clear sky,-24.7911,-50.0119,CHACARA BAILLY
20,Carnarvon,AU,78.87,clear sky,-24.8667,113.6333,Hospitality Carnarvon
...,...,...,...,...,...,...,...
658,Kieta,PG,78.82,moderate rain,-6.2167,155.6333,Uruna Bay Retreat
659,Chake Chake,TZ,79.74,overcast clouds,-5.2459,39.7666,Hotel Archipelago
665,Dingle,PH,76.46,scattered clouds,10.9995,122.6711,SEGAYA PROPERTY
668,Rustenburg,ZA,72.86,broken clouds,-25.6676,27.2421,Rustenburg Boutique Hotel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "weatherpy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{HotelName}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))